In [0]:
# Inv Setup
from google.colab import drive
drive.mount('/content/drive')


In [0]:
tensorflow-gpu--version

In [0]:
# Import Data from Drive
%cp "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/Alexnet/Firefront_Dataset.zip" "/content/Firefront_Dataset.zip"
%cp "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/Alexnet/AlexNet Fire detection.ipynb" "/content/Fire detection.ipynb"
%cd /content/
!unzip Firefront_Dataset.zip


In [0]:
!pip3 install keras
!pip3 install tensorflow-gpu==1.14

In [0]:
#Data setup for training
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="Firefront_Dataset/train_set",target_size=(227,227))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="Firefront_Dataset/test_set", target_size=(227,227))

Using TensorFlow backend.


Found 1750 images belonging to 2 classes.
Found 703 images belonging to 2 classes.


In [0]:
#Definition of network Structure
import keras
import matplotlib.pyplot as plt
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), strides=(4,4), padding="valid", activation = "relu"))

# Max Pooling
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding="same", activation = "relu"))

# Max Pooling
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same", activation = "relu"))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same", activation = "relu"))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="same", activation = "relu"))

# Max Pooling
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(units = 9216, activation = "relu"))

# 2nd Fully Connected Layer
model.add(Dense(units = 4096, activation = "relu"))

# 3rd Fully Connected Layer
model.add(Dense(4096, activation = "relu"))

# Output Layer
model.add(Dense(2, activation = "softmax")) #As we have two classes

In [0]:
#Cost and optimization function

#from keras.optimizers import Adam
#opt = Adam(lr=0.001)
#model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

from keras.optimizers import SGD,Adam,Adadelta,Adagrad,RMSprop
opt = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

#keras.optimizers.Adagrad(lr=0.01, epsilon=1e-08, decay=0.0)
#keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
#keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
#keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

In [4]:
# Training
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("alexnet_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit_generator(steps_per_epoch=50,generator=traindata, validation_data= testdata, validation_steps=50,epochs=100,callbacks=[checkpoint,early])


NameError: ignored

In [0]:
# Plot Performance DAta
import matplotlib.pyplot as plt
plt.plot(hist.history["acc"])
#plt.plot(hist.history['val_acc'])
#plt.plot(hist.history['loss'])
#plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
#plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [0]:
#Try an image for detection
!wget "https://cdn02.carsforsale.com/3/540916/14715988/968420067.jpg"

In [0]:
#Show result of detection for image
from keras.preprocessing import image
img = image.load_img("corsican_image1.png",target_size=(227,227))
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis=0)
from keras.models import load_model
saved_model = load_model("alexnet_1.h5")
output = saved_model.predict(img)
if output[0][0] > output[0][1]:
    print("fire")
else:
    print('no fire')

In [0]:
!unzip Firefront_Dataset1.zip